In [6]:
import pandas as pd
import joblib

# Cargar el modelo entrenado
model = joblib.load('../Modelos_ML/random_forest_regressor.joblib')

# Cargar el dataset principal
df_costos = pd.read_csv('../Data/car_resale_prices_clean.csv')

# Cargar el dataset de costos operacionales
df = pd.read_csv('../Data/costo_operacional_vehiculos_clean.csv', sep=',')

# Calcular umbrales de costos para clasificación
low_cost_threshold = df_costos['resale_price'].quantile(0.33)
high_cost_threshold = df_costos['resale_price'].quantile(0.66)

# Función para categorizar los vehículos en df_costos
def categorize_vehicle(row):
    if row['fuel_type'] in ['Diesel', 'Petrol', 'Petrol/LPG']:
        return 'Convencional'
    elif row['fuel_type'] in ['Electricity', 'Electric']:
        return 'Eléctrico'
    else:
        return 'Híbrido'
    
# Función para categorizar los vehículos en df_costos
def categorize_vehicle2(row):
    if row['Fuel_Type'] in ['Diesel', 'Petrol', 'Petrol/LPG']:
        return 'Convencional'
    elif row['Fuel_Type'] in ['Electricity', 'Electric']:
        return 'Eléctrico'
    else:
        return 'Híbrido'

# Aplicar la función al dataframe
df_costos['Vehicle_Type'] = df_costos.apply(categorize_vehicle, axis=1)
# Aplicar la función al dataframe
df['Vehicle_Type'] = df.apply(categorize_vehicle2, axis=1)

# Función para clasificar el costo
def clasificar_costo(total_cost):
    if total_cost <= low_cost_threshold:
        return 'Económico'
    elif total_cost <= high_cost_threshold:
        return 'Normal'
    else:
        return 'Caro'

# Agregar la columna de categoría de costo al dataset de costos
df_costos['Categoria_Costo'] = df_costos['resale_price'].apply(clasificar_costo)

# Función para calcular el costo operativo estimado
def calcular_costo_operativo(tipo_combustible):
    tipo_combustible = tipo_combustible.capitalize()
    if tipo_combustible in df['Fuel_Type'].unique():
        datos = df[df['Fuel_Type'] == tipo_combustible]
        costo_promedio = df['Fuel_Cost'].mean()
        contaminacion_sonora_promedio = df['Noise_Level'].mean()
        return round(costo_promedio, 2), round(contaminacion_sonora_promedio, 2)
    return 0, 0

# Simulación de la entrada del presupuesto
presupuesto_cliente = 5000  # Cambia este valor manualmente en el notebook

# Filtrar los autos dentro del presupuesto
autos_recomendados = df_costos[df_costos['resale_price'] <= presupuesto_cliente].sort_values(by='resale_price').head(5)

# Si hay autos que cumplen el criterio
if not autos_recomendados.empty:
    print('Autos recomendados dentro de tu presupuesto:')
    print(autos_recomendados[['full_name', 'registered_year', 'fuel_type', 'resale_price']].rename(
        columns={
            'full_name': 'Nombre Completo',
            'registered_year': 'Año',
            'fuel_type': 'Tipo de Combustible',
            'resale_price': 'Precio en Dólares'
        }
    ).round({'Precio en Dólares': 2}))
else:
    print('No se encontraron autos dentro de tu presupuesto.')

# Simular selección de un auto (elige un auto por índice)
selected_auto_index = 0  # Cambia este valor según la opción deseada

if not df_costos.empty:
    selected_auto = df_costos.iloc[selected_auto_index]

    print("\nDetalles del auto seleccionado:")
    print(f"Nombre Completo: {selected_auto['full_name']}")
    print(f"Año: {selected_auto['registered_year']}")
    print(f"Tipo de Combustible: {selected_auto['fuel_type']}")
    print(f"Precio en Dólares: {selected_auto['resale_price']}")
    print(f"Clasificación: {selected_auto['Categoria_Costo']}")

    # Calcular el costo operativo y la contaminación sonora
    tipo_combustible = selected_auto['fuel_type']
    costo_operativo, contaminacion_sonora = calcular_costo_operativo(tipo_combustible)

    print(f'Costo operativo estimado por cada 10,000 km para tipo de combustible "{tipo_combustible}": ${costo_operativo}')
    print(f'Contaminación sonora promedio para tipo de combustible "{tipo_combustible}": {contaminacion_sonora} dB')


Autos recomendados dentro de tu presupuesto:
                     Nombre Completo     Año Tipo de Combustible  \
4456     2010 Chevrolet Spark 1.0 LS  2010.0              Petrol   
12702    2006 Hyundai Santro Xing XO  2006.0              Petrol   
12694           2008 Maruti Alto LXi  2008.0              Petrol   
15230         2011 Chevrolet Beat LS  2011.0              Petrol   
12657  2006 Maruti Swift 1.3 VXI ABS  2006.0              Petrol   

       Precio en Dólares  
4456              1200.0  
12702             1200.0  
12694             1200.0  
15230             1200.0  
12657             1200.0  

Detalles del auto seleccionado:
Nombre Completo: 2017 Maruti Baleno 1.2 Alpha
Año: 2017.0
Tipo de Combustible: Petrol
Precio en Dólares: 6540.0
Clasificación: Normal
Costo operativo estimado por cada 10,000 km para tipo de combustible "Petrol": $1383.38
Contaminación sonora promedio para tipo de combustible "Petrol": 64.43 dB


In [7]:
print(df_costos['resale_price'].head(10))


0     6540.0
1    12000.0
2     5400.0
3     5400.0
4     1920.0
5     5640.0
6     4320.0
7     2400.0
8     4440.0
9     1440.0
Name: resale_price, dtype: float64


In [8]:
import pandas as pd

# Simulación de la entrada del presupuesto
presupuesto_cliente = 5000  # Cambia este valor manualmente en el notebook

# Mostrar los precios antes de cualquier filtrado
print("Precios antes del filtrado:")
print(df_costos[['full_name', 'resale_price']].head(10))

# Filtrar los autos dentro del presupuesto
autos_recomendados = df_costos[df_costos['resale_price'] <= presupuesto_cliente].sort_values(by='resale_price').head(5)

# Mostrar los autos recomendados y verificar que los precios sean correctos
print("\nAutos recomendados (con precios) después del filtrado:")
print(autos_recomendados[['full_name', 'resale_price']])

# Si hay autos que cumplen el criterio
if not autos_recomendados.empty:
    print('\nAutos recomendados dentro de tu presupuesto:')
    print(autos_recomendados[['full_name', 'registered_year', 'Vehicle_Type', 'resale_price']].rename(
        columns={
            'full_name': 'Nombre Completo',
            'registered_year': 'Año',
            'Vehicle_Type': 'Tipo de Combustible',
            'resale_price': 'Precio en Dólares'
        }
    ).round({'Precio en Dólares': 2}))
else:
    print('No se encontraron autos dentro de tu presupuesto.')


Precios antes del filtrado:
                                 full_name  resale_price
0             2017 Maruti Baleno 1.2 Alpha        6540.0
1                       2018 Tata Hexa XTA       12000.0
2              2015 Maruti Swift Dzire VXI        5400.0
3              2015 Maruti Swift Dzire VXI        5400.0
4               2009 Hyundai i10 Magna 1.1        1920.0
5              2015 Hyundai i20 Active 1.2        5640.0
6         2017 Tata Tiago 1.05 Revotorq XZ        4320.0
7               2010 Hyundai i10 Magna 1.1        2400.0
8  2016 Maruti Eeco 7 Seater Standard BSIV        4440.0
9              2009 Hyundai Santro Xing GL        1440.0

Autos recomendados (con precios) después del filtrado:
                           full_name  resale_price
4456     2010 Chevrolet Spark 1.0 LS        1200.0
12702    2006 Hyundai Santro Xing XO        1200.0
12694           2008 Maruti Alto LXi        1200.0
15230         2011 Chevrolet Beat LS        1200.0
12657  2006 Maruti Swift 1.3 VXI A

In [9]:
# Mostrar los primeros autos recomendados sin límite
autos_recomendados_sin_limite = df_costos[df_costos['resale_price'] <= presupuesto_cliente].sort_values(by='resale_price')

# Mostrar los autos antes de aplicar el límite de 5
print("\nAutos recomendados antes de limitar a 5:")
print(autos_recomendados_sin_limite[['full_name', 'resale_price']])

# Aplicar el límite de 5 autos recomendados
autos_recomendados = autos_recomendados_sin_limite.head(5)

# Mostrar los autos recomendados con el límite aplicado
print("\nAutos recomendados después de limitar a 5:")
print(autos_recomendados[['full_name', 'resale_price']])



Autos recomendados antes de limitar a 5:
                                            full_name  resale_price
4456                      2010 Chevrolet Spark 1.0 LS        1200.0
12702                     2006 Hyundai Santro Xing XO        1200.0
12694                            2008 Maruti Alto LXi        1200.0
15230                          2011 Chevrolet Beat LS        1200.0
12657                   2006 Maruti Swift 1.3 VXI ABS        1200.0
...                                               ...           ...
5295            2019 Maruti Celerio X VXI Option BSIV        4992.0
15354                2017 Maruti Swift LXI Optional-O        4992.0
7480         2016 Ford Aspire 1.2 Ti-VCT Ambiente ABS        4992.0
3795                          2016 Maruti Celerio ZXI        4992.0
13550  2017 Hyundai Grand i10 1.2 Kappa Sportz Option        4992.0

[5015 rows x 2 columns]

Autos recomendados después de limitar a 5:
                           full_name  resale_price
4456     2010 Chevrole

In [12]:
presupuesto_cliente = 4000
# Ordenar correctamente por 'resale_price'
autos_recomendados_sin_limite = df_costos[df_costos['resale_price'] <= presupuesto_cliente].sort_values(by='resale_price', ascending=False)

# Mostrar los primeros autos antes de limitar a 5
print("\nAutos recomendados antes de limitar a 5:")
print(autos_recomendados_sin_limite[['full_name', 'resale_price']].head(10))

# Aplicar el límite de 5 autos recomendados
autos_recomendados = autos_recomendados_sin_limite.head(5)

# Mostrar los autos recomendados con el límite aplicado
print("\nAutos recomendados después de limitar a 5:")
print(autos_recomendados[['full_name', 'resale_price']])



Autos recomendados antes de limitar a 5:
                            full_name  resale_price
11996     2013 Maruti Wagon R LXI CNG        3996.0
13834  2011 Hyundai Verna 1.6 EX VTVT        3996.0
9619     2018 Maruti Alto 800 LXI CNG        3996.0
4114         2018 Maruti Alto K10 VXI        3996.0
866          2018 Maruti Alto K10 VXI        3996.0
8004            2011 Maruti Swift LXI        3996.0
1812     2016 Hyundai i10 Sportz 1.1L        3984.0
7061         2016 Maruti Alto K10 VXI        3984.0
10649            2012 Honda Brio S MT        3984.0
7734          2012 Hyundai i10 Sportz        3984.0

Autos recomendados después de limitar a 5:
                            full_name  resale_price
11996     2013 Maruti Wagon R LXI CNG        3996.0
13834  2011 Hyundai Verna 1.6 EX VTVT        3996.0
9619     2018 Maruti Alto 800 LXI CNG        3996.0
4114         2018 Maruti Alto K10 VXI        3996.0
866          2018 Maruti Alto K10 VXI        3996.0


In [11]:
# Revisa la función de categorización de vehículos
def categorize_vehicle(row):
    if row['Fuel_Type'] in ['Diesel', 'Petrol', 'Petrol/LPG']:
        return 'Convencional'
    elif row['Fuel_Type'] in ['Electricity', 'Electric']:
        return 'Eléctrico'
    else:
        return 'Híbrido'

# Aplicar la función al dataframe
df_costos['Vehicle_Type'] = df_costos.apply(categorize_vehicle, axis=1)

# Mostrar autos recomendados dentro del presupuesto
if presupuesto_cliente > 0:
    st.write(f'Autos recomendados dentro de tu presupuesto de ${presupuesto_cliente}:')
    
    # Filtrar los autos dentro del presupuesto
    autos_recomendados = df_costos[df_costos['Resale_Price'] <= presupuesto_cliente].sort_values(by='Resale_Price', ascending=False).head(5)
    
    # Si hay autos que cumplen el criterio
    if not autos_recomendados.empty:
        st.write(autos_recomendados[['Full_Name', 'Registered_Year', 'Fuel_Type', 'Resale_Price', 'Vehicle_Type']].rename(
            columns={
                'Full_Name': 'Nombre Completo',
                'Registered_Year': 'Año',
                'Fuel_Type': 'Tipo de Combustible',
                'Resale_Price': 'Precio en Dólares',
                'Vehicle_Type': 'Clasificación de Combustible'
            }
        ).round({'Precio en Dólares': 2}))
    else:
        st.write('No se encontraron autos dentro de tu presupuesto.')

# Streamlit App
st.title('Evaluación de Autos')

# Entrada del presupuesto por parte del cliente
presupuesto_cliente = st.number_input('Ingresa tu presupuesto (en dólares):', min_value=0)

# Mostrar autos recomendados dentro del presupuesto
if presupuesto_cliente > 0:
    st.write(f'Autos recomendados dentro de tu presupuesto de ${presupuesto_cliente}:')
    
    # Filtrar los autos dentro del presupuesto
    autos_recomendados = df_costos[df_costos['resale_price'] <= presupuesto_cliente].sort_values(by='Resale_Price', ascending=False).head(5)
    
    # Si hay autos que cumplen el criterio
    if not autos_recomendados.empty:
        st.write(autos_recomendados[['Full_Name', 'Registered_Year', 'Fuel_Type', 'Resale_Price', 'Vehicle_Type']].rename(
            columns={
                'Full_Name': 'Nombre Completo',
                'Registered_Year': 'Año',
                'Fuel_Type': 'Tipo de Combustible',
                'Resale_Price': 'Precio en Dólares',
                'Vehicle_Type': 'Clasificación de Combustible'
            }
        ).round({'Precio en Dólares': 2}))
    else:
        st.write('No se encontraron autos dentro de tu presupuesto.')

# Mostrar detalles del auto seleccionado
st.write('Detalles del auto seleccionado:')
autos_list = df_costos['Full_Name'].unique()
selected_auto = st.selectbox('Selecciona un auto:', autos_list)

if selected_auto:
    auto_data = df_costos[df_costos['Full_Name'] == selected_auto]
    st.write(auto_data[['Full_Name', 'Registered_Year', 'Fuel_Type', 'Resale_Price', 'Vehicle_Type']].rename(
        columns={
            'Full_Name': 'Nombre Completo',
            'Registered_Year': 'Año',
            'Fuel_Type': 'Tipo de Combustible',
            'Resale_Price': 'Precio en Dólares',
            'Vehicle_Type': 'Clasificación de Combustible'
        }
    ).round({'Precio en Dólares': 2}))


2024-09-16 09:26:12.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:12.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:12.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:12.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:13.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:13.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:13.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-16 09:26:13.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

# Para revisar en el ETL 1

In [5]:
import pandas as pd

df_Vehiculos = pd.read_csv('../sources/Vehicle Fuel Economy Data.csv')
# Ver los valores únicos de la columna 'Fuel'
valores_unicos_fuel = df_Vehiculos['fuelType2'].unique()
print(valores_unicos_fuel)


[nan 'E85' 'Electricity' 'Natural Gas' 'Propane']


C:\Users\Familia\AppData\Local\Temp\ipykernel_1836\3454241528.py:3: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Vehiculos = pd.read_csv('../sources/Vehicle Fuel Economy Data.csv')


In [6]:
import pandas as pd

df_Vehiculos = pd.read_csv('../sources/Vehicle Fuel Economy Data.csv')
# Ver los valores únicos de la columna 'Fuel'
valores_unicos_fuel = df_Vehiculos['atvType'].unique()
print(valores_unicos_fuel)

[nan 'Hybrid' 'Diesel' 'EV' 'FFV' 'Plug-in Hybrid' 'CNG' 'Bifuel (CNG)'
 'Bifuel (LPG)']


C:\Users\Familia\AppData\Local\Temp\ipykernel_1836\1255093919.py:3: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Vehiculos = pd.read_csv('../sources/Vehicle Fuel Economy Data.csv')


# Revisar en esta funcion de abajo en agregarle todos los convencionales y electricos, el resto simplemente seran hibridos

In [ ]:
# Función para categorizar los vehículos
def categorize_vehicle(row):
    if row['Fuel_Type'] in ['Diesel', 'Petrol', 'Petrol/LPG']:
        return 'Convencional'
    elif row['Fuel_Type'] in ['Electricity', 'Electric']:
        return 'Eléctrico'
    else:
        return 'Híbrido'

# Aplicar la función al dataframe
df['Vehicle_Type'] = df.apply(categorize_vehicle, axis=1)

# Verificar la distribución
print(df['Vehicle_Type'].value_counts())